In [8]:
# pip install pillow requests numpy

import io, os, math, requests
import numpy as np
from PIL import Image


In [9]:

# ---- Config ----
E_MAG = 0.30             # shear magnitude for e1/e2 variants
CANVAS = (512, 512)      # output canvas size
OUTDIR = "twemoji_shears"
os.makedirs(OUTDIR, exist_ok=True)

# Twemoji "🙂" (U+1F642) asset (pick PNG 72x72; could also use SVG)
TWE_URL = "https://cdnjs.cloudflare.com/ajax/libs/twemoji/14.0.2/72x72/1f642.png"


def download_png(url: str) -> Image.Image:
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    return Image.open(io.BytesIO(r.content)).convert("RGBA")

def paste_center(img: Image.Image, size=CANVAS) -> Image.Image:
    """Paste image centered on transparent canvas, keeping original aspect."""
    canvas = Image.new("RGBA", size, (255, 255, 255, 0))
    # scale emoji up a bit to look nice
    scale = min(size[0] * 0.75 / img.width, size[1] * 0.75 / img.height)
    new_w = max(1, int(img.width * scale))
    new_h = max(1, int(img.height * scale))
    resized = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
    x = (size[0] - new_w) // 2
    y = (size[1] - new_h) // 2
    canvas.alpha_composite(resized, (x, y))
    return canvas

def affine_from_e(e1, e2, center):
    """Return PIL affine inverse coefficients for shear-like mapping:
       [x'] = [1+e1  e2 ] [x]
       [y']   [ e2  1-e1] [y]
    about 'center'.
    """
    cx, cy = center
    M = np.array([[1+e1, e2,   0],
                  [e2,   1-e1, 0],
                  [0,    0,    1]], dtype=float)
    T1 = np.array([[1,0,-cx],[0,1,-cy],[0,0,1]], dtype=float)
    T2 = np.array([[1,0, cx],[0,1, cy],[0,0,1]], dtype=float)
    A = T2 @ M @ T1
    Ainv = np.linalg.inv(A)
    a, b, c = Ainv[0,0], Ainv[0,1], Ainv[0,2]
    d, e, f = Ainv[1,0], Ainv[1,1], Ainv[1,2]
    return (a,b,c,d,e,f)

def apply_shear(img: Image.Image, e1=0.0, e2=0.0) -> Image.Image:
    cx, cy = img.size[0]/2, img.size[1]/2
    coeffs = affine_from_e(e1, e2, (cx, cy))
    return img.transform(
        img.size,
        Image.Transform.AFFINE,
        coeffs,
        resample=Image.Resampling.BICUBIC,
        fillcolor=(255,255,255,0)
    )


In [10]:
# --- Run ---
emoji = download_png(TWE_URL)
base = paste_center(emoji, CANVAS)
base_path = os.path.join(OUTDIR, "emoji_twemoji_base.png")
base.save(base_path)

variants = {
    "emoji_twemoji_e1_pos.png": ( E_MAG,  0.0),
    "emoji_twemoji_e1_neg.png": (-E_MAG,  0.0),
    "emoji_twemoji_e2_pos.png": ( 0.0,    E_MAG),
    "emoji_twemoji_e2_neg.png": ( 0.0,   -E_MAG),
}

paths = {"base": base_path}
for fname, (e1, e2) in variants.items():
    out = apply_shear(base, e1=e1, e2=e2)
    path = os.path.join(OUTDIR, fname)
    out.save(path)
    paths[fname] = path

print("Saved files:")
for k,v in paths.items():
    print(f"  {k}: {v}")

Saved files:
  base: twemoji_shears/emoji_twemoji_base.png
  emoji_twemoji_e1_pos.png: twemoji_shears/emoji_twemoji_e1_pos.png
  emoji_twemoji_e1_neg.png: twemoji_shears/emoji_twemoji_e1_neg.png
  emoji_twemoji_e2_pos.png: twemoji_shears/emoji_twemoji_e2_pos.png
  emoji_twemoji_e2_neg.png: twemoji_shears/emoji_twemoji_e2_neg.png
